# Make Transparent Sepecific Color using CV2

In [ ]:
# Libraries

import cv2 as cv
import numpy as np
# import time as t

In [ ]:
# Pre Recording Env

def preRecord(cap):
  for _ in range(30):
    ret, background = cap.read()
    if not ret:
      print("Can't capture background frame.")
      return
  return background

In [ ]:
# Main Function

def run_app():
    cap = cv.VideoCapture(0)
    if not cap.isOpened():
        print("Camera not accessible.")
        return

    ######### Pre-Recorded BG #########
    background = preRecord(cap)
    background = cv.flip(background, 1)

    ######### Main #############
    default_color = "blue"

    hsvRanges = {
        "blue": (np.array([90,50,50]), np.array([130,255,255])),
    }

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        frame = cv.flip(frame, flipCode=1)
        # gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        blur = cv.medianBlur(frame, 5)
        cvtHSV = cv.cvtColor(blur, cv.COLOR_BGR2HSV)

        lower, upper = hsvRanges[default_color]
        mask = cv.inRange(cvtHSV, lower, upper)

        kernel = np.ones((7, 7), np.uint8)
        mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
        mask = cv.morphologyEx(mask, cv.MORPH_DILATE, kernel)

        inv_mask = cv.bitwise_not(mask)
        background_part = cv.bitwise_and(background, background, mask=mask)
        foreground_part = cv.bitwise_and(frame, frame, mask=inv_mask)

        final = cv.addWeighted(background_part, 1, foreground_part, 1, 0)
        cv.imshow("Invisibility Effect", final)

        if cv.waitKey(1) & 0xFF == ord('q'): # Press 'q' to exit
            break

    cap.release()
    cv.destroyAllWindows()

In [ ]:
# Call Function

if __name__ == "__main__":
    run_app()